In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager # sustituye al archivo
from selenium.webdriver.chrome.options import Options # opciones de chrome
from selenium.webdriver.common.by import By # By es para buscar por tag, clase, id...
from selenium.webdriver.support.ui import WebDriverWait   # para meter esperaras
from selenium.webdriver.support import expected_conditions as EC   # para esperar ciertos eventos
from selenium.webdriver import ActionChains # para hacer acciones con el ratón
from selenium.webdriver.support import expected_conditions as EC   # para esperar ciertos eventos
from selenium.webdriver.chrome.service import Service
import time
import selenium.webdriver.common.keys as Keys # para simular teclas
import pandas as pd

In [ ]:
#Opciones de chrome
opciones = webdriver.ChromeOptions()
opciones.add_argument('--start-maximized')
opciones.add_argument('--disable-extensions')
opciones.add_argument('--disable-blink-features=AutomationControlled')
opciones.add_argument('--no-sandbox')
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_argument("--headless")

# guardar las cookies
opciones.add_argument('--incognito') # modo incognito
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'

In [ ]:
url = "https://anper.net/productos/destacados"
driver = webdriver.Chrome(options=opciones)
driver.get(url)

In [ ]:
# recojo todos los elementos de la lista de categorias
# guardo solamente aquellas que no tienen "collapse" en la url, ya que me llevan a menús repetidos
selector = driver.find_element(By.ID, "accordion")
lista = selector.find_elements(By.TAG_NAME, "a")
lista_limpia = []
for e in lista:
    enlace = e.get_attribute("href")
    if "collapse" in enlace:
       print(f"{enlace} eliminado")
    else:
       lista_limpia.append(e.get_attribute("href"))

In [ ]:
#defino el driver e inicializo la lista donde voy a guardar los productos

driver2 = webdriver.Chrome(options=opciones)
productos_anper = []
#recorro la lista de categorías para acceder a cada producto
for e in lista_limpia:
    #inicializo el driver y recojo los productos de la página en una lista
    print(e)
    driver2.get(e)
    lista_productos = driver2.find_elements(By.CLASS_NAME, "item.equalheight")
    #recorro la lista de productos para coger la información que necesito de cada producto
    for i in range(len(lista_productos)):
        try:
            #recojo la lista de los productos de cada página
            lista_productos = driver2.find_elements(By.CLASS_NAME, "item.equalheight")
            #guardo el atributo href y hago click en cada producto para acceder a las características
            url_producto = lista_productos[i].find_element(By.TAG_NAME, "a").get_attribute("href")
            try: 
                lista_productos[i].find_element(By.TAG_NAME, "a").click()
            except:
                driver2.find_element(By.CLASS_NAME, "cc-cookie-accept").click() 
                lista_productos[i].find_element(By.TAG_NAME, "a").click()
            #intento recoger la categoría y el nombre de los breadcrumbs, 
            # algunas veces da fallo, así que lo recojo de la url de cada producto
            try: 
                breadcrumb = driver2.find_element(By.XPATH, '//*[@id="product"]/div/div[1]/div').text
                lista_breadcrumb = breadcrumb.split(" > ")
            
                if len(lista_breadcrumb) == 4:
                    categoria = lista_breadcrumb[1]
                    subcategoria = lista_breadcrumb[2]
                    nombre = lista_breadcrumb[3]
                else:
                    categoria = lista_breadcrumb[1]
                    subcategoria = "no tiene"
                    nombre = lista_breadcrumb[2]
            except:
                url_lista = url_producto.split("/")
                categoria = url_lista[-2]
                subcategoria= "no tiene"
                nombre = url_lista[-1]

            #recojo el precio, en productos en los que tengo varios precios, 
            # guardo la lista y lo tratará en el proceso de limpieza
            try: 
                precio = driver2.find_element(By.CLASS_NAME, "price").text
            except:
                precio = driver2.find_element(By.CLASS_NAME, "selectrOptions").text

            descripcion = driver2.find_element(By.CLASS_NAME, "text").text
            # lo almaceno todo en un diccionario
            producto_dicc ={
                "categoria" : categoria,
                "subcategoria" : subcategoria,
                "nombre" : nombre,
                "precio" : precio,
                "url" : url_producto,
                "descripcion" : descripcion,
                "tienda" : "Anper"
            }
            #almaceno cada diccionario en la lista antes de volver atrás para almacenar el siguiente producto
            productos_anper.append(producto_dicc)
            time.sleep(2)
            driver2.back()
        except: 
            print(f'el producto: {url_producto} da error')

In [ ]:
# Crear un DataFrame con la lista de productos_info
df_productos = pd.DataFrame(productos_anper)

# Guardar el DataFrame en un archivo CSV
df_productos.to_csv("anper.csv", index=False)